In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import gc

zepa = '/content/gdrive/MyDrive/NF_ToN_IoT_v2_full170chunks'

os.chdir(zepa)
parquet_files = [filename for filename in os.listdir(zepa) if filename.endswith('.parquet')]
parquet_files = parquet_files[:-1]
print(parquet_files)
print(len(parquet_files))

['NF-ToN-IoT-v2_CH_0.parquet', 'NF-ToN-IoT-v2_CH_1.parquet', 'NF-ToN-IoT-v2_CH_2.parquet', 'NF-ToN-IoT-v2_CH_3.parquet', 'NF-ToN-IoT-v2_CH_4.parquet', 'NF-ToN-IoT-v2_CH_5.parquet', 'NF-ToN-IoT-v2_CH_6.parquet', 'NF-ToN-IoT-v2_CH_7.parquet', 'NF-ToN-IoT-v2_CH_8.parquet', 'NF-ToN-IoT-v2_CH_9.parquet', 'NF-ToN-IoT-v2_CH_10.parquet', 'NF-ToN-IoT-v2_CH_11.parquet', 'NF-ToN-IoT-v2_CH_12.parquet', 'NF-ToN-IoT-v2_CH_13.parquet', 'NF-ToN-IoT-v2_CH_14.parquet', 'NF-ToN-IoT-v2_CH_15.parquet', 'NF-ToN-IoT-v2_CH_16.parquet', 'NF-ToN-IoT-v2_CH_17.parquet', 'NF-ToN-IoT-v2_CH_18.parquet', 'NF-ToN-IoT-v2_CH_19.parquet', 'NF-ToN-IoT-v2_CH_20.parquet', 'NF-ToN-IoT-v2_CH_21.parquet', 'NF-ToN-IoT-v2_CH_22.parquet', 'NF-ToN-IoT-v2_CH_23.parquet', 'NF-ToN-IoT-v2_CH_24.parquet', 'NF-ToN-IoT-v2_CH_25.parquet', 'NF-ToN-IoT-v2_CH_26.parquet', 'NF-ToN-IoT-v2_CH_27.parquet', 'NF-ToN-IoT-v2_CH_28.parquet', 'NF-ToN-IoT-v2_CH_29.parquet', 'NF-ToN-IoT-v2_CH_30.parquet', 'NF-ToN-IoT-v2_CH_31.parquet', 'NF-ToN-IoT-v2_CH

In [ ]:
import pickle
import numpy as np
import scipy.stats as statists
from scipy.stats import skew, kurtosis
from scipy.stats import boxcox
from scipy.stats import yeojohnson
directory = '/content/gdrive/MyDrive/NF_ToN_IoT_v2_full170chunks'
varta = [4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31,
         32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
daList = ['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT','PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS','TCP_FLAGS',
          'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS','FLOW_DURATION_MILLISECONDS', 'DURATION_IN', 'DURATION_OUT', 'MIN_TTL','MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT',
          'MIN_IP_PKT_LEN','MAX_IP_PKT_LEN', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES','RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS','RETRANSMITTED_OUT_BYTES',
          'RETRANSMITTED_OUT_PKTS','SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT','NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES','NUM_PKTS_256_TO_512_BYTES',
          'NUM_PKTS_512_TO_1024_BYTES','NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT','ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
          'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label', 'Attack']

In [ ]:
import random
import math

def generate_unique_values(a_range, a_count): # generates a list of indices for sampling
    # for NF_ToN_IoT-v2 data, a_range = 16940496
    if a_count > a_range:
        raise ValueError("Count cannot exceed range")
    unique_values = random.sample(range(a_range), a_count)
    return np.array(unique_values)

def give_all_indices(a_range:int,a_count:int,z_count,tvt_split=(0.7,0.15,0.15)):# indices come out as 1)z_set,2)train,3)validation,4)test sets
    '''
      a_range= # of rows in dataset
      a_count--full count of what is to be pulled
      z_count -- count of rows for which mean, s.d. and z-score are to be calc'd
      tvt_split -- train, val, test by decimal fraction
    '''
    full_set = generate_unique_values(a_range,a_count)
    z_set = full_set[0:z_count]
    full_set = full_set[z_count:]
    tvt_set_count = a_count - z_count
    train_count = int(tvt_split[0]*tvt_set_count)
    valid_count = int(tvt_split[1]*tvt_set_count)
    train_set = full_set[0:train_count]
    full_set = full_set[train_count:]
    valid_set = full_set[0:valid_count]
    test_set = full_set[valid_count:]
    return z_set,train_set,valid_set,test_set

In [ ]:

def open_pkl(a_path,column_no, a_tail):
  outPut_filePath = a_path + "/ton_iot_" + daList[varta[column_no]] + a_tail
  with open(outPut_filePath, 'rb') as f:
    a_work_column = pickle.load(f)
    print(a_work_column.shape,daList[varta[column_no]])
    return a_work_column


def save_pkl(a_path,column_no,this_data,a_tail):# e.g. a_tail = '_mod.pkl'
  modFileName = a_path + "/ton_iot_" + daList[varta[column_no]] + a_tail
  with open(modFileName, 'wb') as f:
    pickle.dump(this_data,f)

In [ ]:
# GROUP I FUNCTIONS
def binary_me(incoming:np.array) -> np.array : # for column 33 DNS_QUERY_ID
  return np.array([item if item == 0 else 1 for item in incoming])
def normalizer(incoming:np.array) -> np.array :
  a_min = np.min(incoming)
  a_max = np.max(incoming)
  intermed = [(x - a_min) / (a_max - a_min) for x in incoming]
  return np.array(intermed)
def stack_norm(incoming:np.array) -> np.array :
  intermed = normalizer(incoming)
  return np.vstack((incoming,intermed))
def stack_bi(incoming:np.array) -> np.array :
  intermed = binary_me(incoming)
  return np.vstack((incoming,intermed))
def sort_grpI(incoming:np.array,sele:int)->np.array:
  if sele == 0:
    return stack_norm(incoming)
  return stack_bi(incoming)


PROTOCOL 0
TCP_FLAGS 6
CLIENT_TCP_FLAGS 7
SERVER_TCP_FLAGS 8
MIN_TTL 12
MAX_TTL 13
ICMP_IV4_TYPE 32
DNS QUERY_ID 33
FTP_COMMAND_RET_CODE 36
L7_PROTO 1

In [ ]:
groupI = [0,6,7,8,12,13,32,36,33,1]
grpI_xfrm = [0,0,0,0,0,0,0,0,1,0]
for p in range(len(groupI)):
  working_column = open_pkl(zepa,groupI[p],'_orig.pkl')
  working_column = sort_grpI(working_column, grpI_xfrm[p])
  save_pkl(zepa,groupI[p],working_column,'_mod.pkl')
  print(working_column[0:5,])
  del working_column
  gc.collect()

(16940496,) PROTOCOL
[[ 6.          6.          1.         ...  6.         17.
   1.        ]
 [ 0.0877193   0.0877193   0.         ...  0.0877193   0.28070175
   0.        ]]
(16940496,) TCP_FLAGS
[[24.         24.          0.         ... 25.          0.
   0.        ]
 [ 0.10762332  0.10762332  0.         ...  0.11210762  0.
   0.        ]]
(16940496,) CLIENT_TCP_FLAGS
[[24.         16.          0.         ... 25.          0.
   0.        ]
 [ 0.10762332  0.07174888  0.         ...  0.11210762  0.
   0.        ]]
(16940496,) SERVER_TCP_FLAGS
[[24.         24.          0.         ... 25.          0.
   0.        ]
 [ 0.10762332  0.10762332  0.         ...  0.11210762  0.
   0.        ]]
(16940496,) MIN_TTL
[[128.         128.          64.         ... 232.           0.
    0.        ]
 [  0.50196078   0.50196078   0.25098039 ...   0.90980392   0.
    0.        ]]
(16940496,) MAX_TTL
[[128.         128.          64.         ... 232.           0.
    0.        ]
 [  0.50196078   0.501960

In [ ]:
# GROUP II FUNCTIONS


def do_sixth(incoming):
  intermed = np.cbrt(incoming)
  return np.sqrt(incoming)
def xform_data(incoming:np.array,this_X:int,chosen_lambda)->np.array:
  if this_X == 0:
    return np.log(incoming)
  elif this_X ==1:
    return np.sqrt(incoming)
  elif this_X == 2:
    return np.cbrt(incoming)
  elif this_X == 3:
    outgo, _ = yeojohnson(incoming)
    return outgo
  elif this_X == 4:
    outgo = statists.boxcox(incoming,lmbda=chosen_lambda)
    outgo[np.isinf(outgo)] = 0
    return outgo
  elif this_X == 5:
    return do_sixth(incoming)
  else:
     outgo, _ = boxcox(incoming)
     return outgo
def stack_xform(incoming:np.array,this_X:int,chosen_lambda):
  intermed = xform_data(incoming,this_X,chosen_lambda)
  return np.vstack((incoming,intermed))


Transformation
Columns
Box Cox
IN-BYTES 2
IN_PKTS 3
LONGEST_FLOW_PKT 14
SHORTEST_FLOW_PKT 15

Manual or “Bespoke” Box Cox Implementation
DST_TO_SRC_SECOND_BYTES 17; lambdas = -3.6, -0.6, -1.9

Yeo-Johnson Transformation
FLOW_DURATION_MILLISECONDS 9

Sixth root
*L7_PROTO 1
OUT_BYTES 4


In [ ]:
groupII = [2,3,14,15,17,9,4]#1 -- L7_PROTO in groupI this round]
grpII_xfrm = [6,6,6,6,4,3,5]
put_lamb = [0,0,0,0,-3.6,0,0]
for p in range(len(groupII)):
  working_column =  open_pkl(zepa,groupII[p],'_orig.pkl')
  working_column = stack_xform(working_column,grpII_xfrm[p],put_lamb[p])
  save_pkl(zepa,groupII[p],working_column,'mod.pkl')
  print(working_column[0:5,])
  del working_column
  gc.collect()


(16940496,) IN_BYTES
[[1.55392000e+05 1.60000000e+03 2.12000000e+02 ... 1.91000000e+02
  7.20000000e+01 1.00000000e+02]
 [2.13336606e+00 2.07312313e+00 1.96589603e+00 ... 1.95713389e+00
  1.85077809e+00 1.89211342e+00]]
(16940496,) IN_PKTS
[[202.          40.           2.         ...   4.           1.
    1.        ]
 [  0.89619627   0.8838216    0.48311074 ...   0.7065004    0.
    0.        ]]
(16940496,) LONGEST_FLOW_PKT
[[1.50000000e+03 1.28600000e+03 1.06000000e+02 ... 7.10000000e+01
  7.20000000e+01 1.00000000e+02]
 [9.46415767e-01 9.46341864e-01 9.39959841e-01 ... 9.36337392e-01
  9.36491310e-01 9.39523485e-01]]
(16940496,) SHORTEST_FLOW_PKT
[[ 40.          40.         106.         ...  40.          72.
  100.        ]
 [  0.43223133   0.43223133   0.43230752 ...   0.43223133   0.4322946
    0.43230623]]
(16940496,) DST_TO_SRC_SECOND_BYTES
[[3.45520000e+04 3.57410000e+04 0.00000000e+00 ... 1.51000000e+02
  0.00000000e+00 0.00000000e+00]
 [2.77777778e-01 2.77777778e-01 0.00000000

In [ ]:


#  GROUP III FUNCTIONS

# Parallel array representing values which truly = 0 and non-zero values
# These arrays get paired on input with processed column
def give_zero_doppel(incoming:np.array,part_on=0.0) -> np.array :
  return np.array([1 if item == 0 else 0 for item in incoming])

# Partitioning function
def pos_partition(incoming:np.array,threshold=0) -> np.array:
  return incoming[incoming>threshold]

# Log -- need to deal with potential 0's; here, implement as 0 = 0 or -1
def process_log(incoming:np.array)-> np.array:
  return np.array([0 if item == 0 else math.log(item)for item in incoming])

# Box-Cox
def push_BC(value,a_lambda):
  if a_lambda == 0:
    return math.log(value)
  return ((value**a_lambda) / a_lambda)

def give_lambda(incoming:np.array)-> float:
  _, lambda_value = boxcox(incoming)
  return lambda_value

def process_bxcx(incoming:np.array,bespoke)-> np.array:
  a_lambda = bespoke
  if bespoke == 0:
    a_lambda = give_lambda(pos_partition(incoming))
  return np.array([0 if item == 0 else push_BC(item, a_lambda) for item in incoming])\

def just_yeo(incoming:np.array)->np.array:
    outgo, _ = yeojohnson(incoming)
    return outgo

def dicem_all(incoming:np.array,a_type:int,bespoke=0.0)-> np.array:
  doppel = give_zero_doppel(incoming)
  passer = np.array([])
  if a_type == 1:
    passer = process_bxcx(incoming,bespoke)
  elif a_type == 2:
    passer = np.array([0 if item == 0 else math.log(item) for item in incoming])
  elif a_type == 3:
    passer = np.sqrt(incoming)
  elif a_type == 4:
    passer = just_yeo(incoming)
  else: passer = np.cbrt(incoming) #a_type = 0
  incoming = np.vstack((incoming,doppel))
  incoming = np.vstack((incoming,passer))
  return incoming











Box Cox
*L7_PROTO 1
OUT_PKTS 5

Cube Root
DURATION_IN 10
DURATION_OUT 11
MIN_IP_PKT_LEN 16
RETRANSMITTED_IN_BYTES 18
RETRANSMITTED_IN_PKTS 19
RETRANSMITTED_OUT_BYTES 20
RETRANSMITTED_OUT_PKTS 21
SRC_TO_DST_AVG_THROUGHPUT 22
DST_TO_SRC_AVG_THROUGHPUT 23
NUM_PKTS_UP_TO_128_BYTES 24
NUM_PKTS_128_TO_256_BYTES 25
NUM_PKTS_256_TO_512 BYTES 26
NUM_PKTS_512_TO_1024_BYTES 27
NUM_PKTS_1024_TO_1514_BYTES 28
TCP_WIN_MAX_IN 29
TCP_WIN_MAX_OUT 30
ICMP_TYPE 31
DNS_TTL_ANSWER 35


In [ ]:
groupIII = [5,10,11,16,18,19,20,21,22,23,24,25,26,27,28,29,31,35,30,34]
grpIII_xfrm = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
for p in range(len(groupIII)):
  working_column =  open_pkl(zepa,groupIII[p],'_orig.pkl')
  working_column = dicem_all(working_column,grpIII_xfrm[p])
  save_pkl(zepa,groupIII[p],working_column,'mod.pkl')
  print(working_column[0:5,])
  del working_column
  gc.collect()


(16940496,) TCP_WIN_MAX_OUT
[[4.80500000e+03 2.37000000e+02 0.00000000e+00 ... 6.38460000e+04
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [3.84005304e+02 5.40783717e+01 0.00000000e+00 ... 2.07071398e+03
  0.00000000e+00 0.00000000e+00]]
(16940496,) DNS_QUERY_TYPE
[[ 0.          0.          0.         ...  0.         12.
   0.        ]
 [ 1.          1.          1.         ...  1.          0.
   1.        ]
 [ 0.          0.          0.         ...  0.          2.28942849
   0.        ]]



y = (x**lmbda - 1) / lmbda,  for lmbda != 0
    log(x),                  for lmbda = 0





In [ ]:
'''
#16940496; seeds = 1066, 1458,1897, 1908,1916, 1935, 1938
the_seed = 1938
random.seed(the_seed)
#a_z_set,a_train_set,a_valid_set,a_test_set = give_all_indices(16940496,1000000,500000)
full_set = give_all_indices(16940496,1000000,500000)
a_tail = str(the_seed) + '.pkl'
modFileName = zepa + "/ton_iot_indices_set" + a_tail


with open(modFileName, 'wb') as f:
  pickle.dump(full_set,f)
'''

In [ ]:
def get_indices_set(a_seed, a_path):
  a_tail = str(a_seed) + '.pkl'
  outPut_filePath = a_path + "/ton_iot_indices_set" + a_tail
  with open(outPut_filePath,'rb') as f:
    full_set = pickle.load(f)
    return full_set



In [ ]:
the_seed = 1938
vaud = get_indices_set(the_seed,zepa)

In [ ]:
z_ind = vaud[0]
train_ind = vaud[1]
valid_ind = vaud[2]
test_ind = vaud[3]
z_ind[0:10],train_ind[0:10],valid_ind[0:10],test_ind[0:10]

(array([ 9713257,  2506646,  9100675, 14020554, 16713656,  4780283,
         1102782,  4376979, 10840477,  2316018]),
 array([ 6086517,  1079215,  5532705,  8579361, 13994048,   822701,
        12928640, 12817112,  5730068, 12604528]),
 array([ 1312379, 10402760,  5910160,  1388138, 11792104,  9615261,
        15561579, 10400733, 13305453,  3842581]),
 array([12267334, 13102615, 11396225,  9794657, 14143356, 14169248,
         7961241,  4082500,  6914253, 14239151]))

In [ ]:
#mean_sd_array = [(0,0) for p in range(37)]


In [ ]:
mean_sd_array = [(0,0) for p in range(37)]
groupIII = [5,10,11,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,34,35]
for p in range(len(groupIII)):
  working_column = open_pkl(zepa,groupIII[p],'mod.pkl')
  working_column = working_column.T
  ind_column = working_column[z_ind,2]
  a_mean = np.mean(ind_column)
  #print(a_mean)
  a_std = np.std(ind_column)
  #print(a_std)
  mean_sd_array[groupIII[p]] = (a_mean,a_std)
  del working_column
  del ind_column
  gc.collect()
print(mean_sd_array)

(3, 16940496) OUT_PKTS
(3, 16940496) DURATION_IN
(3, 16940496) DURATION_OUT
(3, 16940496) MIN_IP_PKT_LEN
(3, 16940496) RETRANSMITTED_IN_BYTES
(3, 16940496) RETRANSMITTED_IN_PKTS
(3, 16940496) RETRANSMITTED_OUT_BYTES
(3, 16940496) RETRANSMITTED_OUT_PKTS
(3, 16940496) SRC_TO_DST_AVG_THROUGHPUT
(3, 16940496) DST_TO_SRC_AVG_THROUGHPUT
(3, 16940496) NUM_PKTS_UP_TO_128_BYTES
(3, 16940496) NUM_PKTS_128_TO_256_BYTES
(3, 16940496) NUM_PKTS_256_TO_512_BYTES
(3, 16940496) NUM_PKTS_512_TO_1024_BYTES
(3, 16940496) NUM_PKTS_1024_TO_1514_BYTES
(3, 16940496) TCP_WIN_MAX_IN
(3, 16940496) TCP_WIN_MAX_OUT
(3, 16940496) ICMP_TYPE
(3, 16940496) DNS_QUERY_TYPE
(3, 16940496) DNS_TTL_ANSWER
[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (-0.7988626186932297, 0.7425521667933648), (0, 0), (0, 0), (0, 0), (0, 0), (0.8835643433998698, 2.2262325637987344), (0.8454258464133698, 2.165808991569393), (0, 0), (0, 0), (0, 0), (0, 0), (2.324945415327987, 1.7939543393368842), (0, 0), (0.13322009000413632, 1.1704050545568665), (

In [ ]:
groupI = [0,6,7,8,12,13,32,36,33,1] # just for completeness; these don't get used.
for p in range(len(groupI)):
  working_column = open_pkl(zepa,groupI[p],'_mod.pkl')
  working_column = working_column.T
  ind_column = working_column[z_ind,1]
  a_mean = np.mean(ind_column)
  a_std = np.std(ind_column)
  mean_sd_array[groupI[p]] = (a_mean,a_std)
  del working_column
  del ind_column
  gc.collect()


print(mean_sd_array)

(2, 16940496) PROTOCOL
(2, 16940496) TCP_FLAGS
(2, 16940496) CLIENT_TCP_FLAGS
(2, 16940496) SERVER_TCP_FLAGS
(2, 16940496) MIN_TTL
(2, 16940496) MAX_TTL
(2, 16940496) ICMP_IPV4_TYPE
(2, 16940496) FTP_COMMAND_RET_CODE
(2, 16940496) DNS_QUERY_ID
(2, 16940496) L7_PROTO
[(0.11606557894736845, 0.06891430999655308), (0.055510837822580636, 0.12899115848047665), (0, 0), (0, 0), (0, 0), (-0.7988626186932297, 0.7425521667933648), (0.057334313901345295, 0.05052728944770073), (0.04249021524663677, 0.04883396187685532), (0.04937372197309418, 0.05170958341351293), (0, 0), (0.8835643433998698, 2.2262325637987344), (0.8454258464133698, 2.165808991569393), (0.09332865882352943, 0.13353621552901154), (0.09341708235294117, 0.13362957185152563), (0, 0), (0, 0), (2.324945415327987, 1.7939543393368842), (0, 0), (0.13322009000413632, 1.1704050545568665), (0.019635896348438465, 0.1664476707321095), (0.2397995777455325, 1.8480473860696733), (0.03431007653322456, 0.2615790576724008), (91.67563634321498, 61.2843

In [ ]:
groupII = [2,3,14,15,17,9,4]
for p in range(len(groupII)):
  working_column = open_pkl(zepa,groupII[p],'mod.pkl')
  working_column = working_column.T
  ind_column = working_column[z_ind,1]
  a_mean = np.mean(ind_column)
  a_std = np.std(ind_column)
  mean_sd_array[groupII[p]] = (a_mean,a_std)
  del working_column
  del ind_column
  gc.collect()
print(mean_sd_array)

(2, 16940496) IN_BYTES
(2, 16940496) IN_PKTS
(2, 16940496) LONGEST_FLOW_PKT
(2, 16940496) SHORTEST_FLOW_PKT
(2, 16940496) DST_TO_SRC_SECOND_BYTES
(2, 16940496) FLOW_DURATION_MILLISECONDS
(2, 16940496) OUT_BYTES
[(0.11606557894736845, 0.06891430999655308), (0.055510837822580636, 0.12899115848047665), (1.8693465298310277, 0.11897089629556083), (0.2599447266369824, 0.35644084198073667), (13.056951443016517, 25.573618965932624), (-0.7988626186932297, 0.7425521667933648), (0.057334313901345295, 0.05052728944770073), (0.04249021524663677, 0.04883396187685532), (0.04937372197309418, 0.05170958341351293), (0.5324886399975564, 1.1189200988738326), (0.8835643433998698, 2.2262325637987344), (0.8454258464133698, 2.165808991569393), (0.09332865882352943, 0.13353621552901154), (0.09341708235294117, 0.13362957185152563), (0.9348740326974966, 0.006456565478840598), (0.43225728591785056, 2.0514731590555735e-05), (2.324945415327987, 1.7939543393368842), (0.17489436130671307, 0.13414062143504663), (0.133

In [ ]:
a_tail = str(the_seed)
mean_std_FileName = zepa + "/ton_iot_mean_std_" + a_tail + '.pkl'
with open(mean_std_FileName, 'wb') as f:
  pickle.dump(mean_sd_array,f)

In [ ]:
def compute_z_scores(data):
    '''For computing Z-scores
    data = array of tuples, each tuple = (mean,s.d.)
    return = array of Z-scores'''

    z_scores = []
    for mean, std_dev in data:

        z_score = (mean - np.mean(data)) / np.std(data)

        z_scores.append(z_score)

    return z_scores


data = [(10, 2), (10000, 5000), (50, 10)]
z_scores = compute_z_scores(data)
print(z_scores)
